In [1]:
# Beautify code
%load_ext nb_black

# Import relevant libraries
import pandas as pd
import numpy as np
import time
import datetime as dt
import warnings

warnings.filterwarnings("ignore")

# start = time.process_time()
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
date_zero = dt.datetime(1900, 1, 1)
date_new_rates = dt.datetime(2018, 7, 1)
date_cft = dt.datetime(2019, 5, 1)
date_co_sponsorship = dt.datetime(2018, 4, 14)
start_date = dt.datetime(2019, 5, 1)
end_date = dt.datetime(2020, 10, 31)
ref_rate = np.array(
    [
        [1, 0.35, 0.2],
        [2, 0.35, 0.3],
        [3, 0.5, 0.4],
        [4, 0.7, 0.5],
        [5, 0.7, 0.5],
        [6, 0.7, 0.5],
        [7, 0.7, 0.5],
        [8, 0.7, 0.5],
    ]
)
position = [
    "Trainee Associate",
    "Associate",
    "Associate Manager",
    "Manager",
    "Senior Manager",
    "Area Manager",
    "District Manager",
    "Director",
]
position_promo = [
    "TA Promo Date",
    "A Promo Date",
    "AM Promo Date",
    "M Promo Date",
    "SM EXEC Promo Date",
    "AM EXEC Promo Date",
    "DM EXEC Promo Date",
    "D EXEC Promo Date",
]
rate_cols = ["IndexNum", "1900-01-01", "2018-07-01"]
referral_rate = pd.DataFrame(ref_rate, position, rate_cols)
referral_rate["IndexNum"] = referral_rate["IndexNum"].astype(int)
gen_override = np.array(
    [
        [1, 0.14, 0.25],
        [2, 0.06, 0.11],
        [3, 0.04, 0.05],
        [4, 0.03, 0.03],
        [5, 0.02, 0.02],
        [6, 0.01, 0.02],
        [7, np.nan, 0.01],
        [8, np.nan, 0.01],
    ]
)
override_rate = pd.DataFrame(gen_override, columns=rate_cols)
override_rate["IndexNum"] = override_rate["IndexNum"].astype(int)
cft_rate = np.array([[1, 0.7], [2, 0.15], [3, 0.05]])
cft = [
    "AA1111",
    "AA1112",
    "AA1117",
    "AA1120",
    "AA1122",  # Timmy Ang, CFT status TBD
    "AA1123",
    "AA1127",
    "AA1128",
    "AA1130",
    "AA1135",  # Shiela Alvarez, CFT status TBD
    "AA1288",
    "AA1323",
    "AA1812",
]

cft_df = pd.DataFrame()

# Soliciting officer
so = "AA1127"

# Company code
company = "AA1110"

# Inactive members
inactive = ["AA1133"]

# Choose which commission to generate
calculate_referral = True

<IPython.core.display.Javascript object>

In [2]:
referral_rate

,IndexNum,1900-01-01,2018-07-01
Trainee Associate,1,0.35,0.2
Associate,2,0.35,0.3
Associate Manager,3,0.50,0.4
Manager,4,0.70,0.5
Senior Manager,5,0.70,0.5
Area Manager,6,0.70,0.5
District Manager,7,0.70,0.5
Director,8,0.70,0.5


<IPython.core.display.Javascript object>

In [3]:
override_rate

,IndexNum,1900-01-01,2018-07-01
0,1,0.14,0.25
1,2,0.06,0.11
2,3,0.04,0.05
3,4,0.03,0.03
4,5,0.02,0.02
5,6,0.01,0.02
6,7,NaN,0.01
7,8,NaN,0.01


<IPython.core.display.Javascript object>

In [4]:
# Read consolidated FAMI commission file
# comm_raw = pd.read_excel(r"Provide_Commission_201801-201904.xlsx", sheet_name="Raw")
# comm_raw = pd.read_excel(r"Provide_Commission_201905-201912.xlsx", sheet_name="Raw")
comm_raw = pd.read_excel("Provide_Commission_201702-202010.xlsx", sheet_name="Raw")

# Convert to dataframe
comm_raw_df = pd.DataFrame(
    comm_raw,
    columns=[
        "CUSTOMER_CODE",
        "SECURITY_CODE",
        "CLIENT_NAME",
        "TRADE_DATE",
        "AMOUNTINVESTED",
        "TRAIL_AMOUNT",
        "SALES_LOAD_RATE",
        "SALES_LOAD_AMOUNT",
        "COMMISSION_LOAD",
        "TRAIL_COMM",
        "TOTAL_COMM",
        "TAXABLE_COMM",
        "WHT",
        "NET_COMM",
        "REPORTDATE",
    ],
)
comm_raw_df.to_excel("comm_raw_df.xlsx")

# Get active accounts
comm_raw_trunc = comm_raw_df[np.isfinite(comm_raw_df["CUSTOMER_CODE"])]
# print(comm_raw_trunc.shape)

# Get initial/additional investments
if calculate_referral:
    calculate_trail = False
    init_addl_inv = comm_raw_trunc["SALES_LOAD_RATE"] > 0
    comm_raw_trunc = comm_raw_trunc[init_addl_inv]
else:
    calculate_trail = True
    init_addl_inv = comm_raw_trunc["TRAIL_AMOUNT"] > 0
    comm_raw_trunc = comm_raw_trunc[init_addl_inv]
# print(comm_raw_trunc.shape)
comm_raw_trunc.to_excel(r"comm_raw_trunc_0.xlsx")

# Convert folio number to string
comm_raw_trunc["CUSTOMER_CODE"] = comm_raw_trunc["CUSTOMER_CODE"].astype(int)
comm_raw_trunc["CUSTOMER_CODE"] = comm_raw_trunc["CUSTOMER_CODE"].astype(str)

comm_raw_trunc["WHT_PCT"] = np.nan
comm_raw_trunc["WHT_PCT"] = comm_raw_trunc["WHT"] / comm_raw_trunc["TAXABLE_COMM"]

# Limit commission data to date range
date_range = (comm_raw_trunc["REPORTDATE"] >= start_date) & (
    comm_raw_trunc["REPORTDATE"] <= end_date
)

# Ad-hoc: Exclude Folio #56895
date_range = date_range & (comm_raw_trunc["CUSTOMER_CODE"] != "56895")

comm_raw_trunc = comm_raw_trunc[date_range]

# Summarize accounts
if calculate_referral:
    comm_raw_trunc = comm_raw_trunc.groupby(
        ["CUSTOMER_CODE", "SECURITY_CODE", "CLIENT_NAME", "SALES_LOAD_RATE",],
        as_index=False,
    ).sum()
else:
    # comm_raw_trunc = comm_raw_trunc.groupby(["CUSTOMER_CODE", "SECURITY_CODE", "CLIENT_NAME", "WHT_PCT"], as_index=False,).sum()
    pass
comm_raw_trunc.to_excel(r"comm_raw_trunc.xlsx")

# Read FAMI Accounts file
accts_raw = pd.read_excel("FAMI_Accounts.xlsx", sheet_name="FAMI")

# Convert to dataframe
accts_raw_df = pd.DataFrame(
    accts_raw,
    columns=[
        "DATE",
        "CLIENT_NAME",
        "SC_CODE",
        "SALES_COUNSELOR",
        "MANAGER",
        "FIRST_PAY",
        "ACCOUNT_CODE",
    ],
)
accts_raw_df["ACCOUNT_CODE"] = accts_raw_df["ACCOUNT_CODE"].astype(str)

# Limit to report date range
# date_range = ((accts_raw_df["DATE"] >= start_date) & (accts_raw_df["DATE"] <= end_date) & (accts_raw_df["ACCOUNT_CODE"] != "56895"))
# accts_raw_trunc = accts_raw_df[date_range]
accts_raw_trunc = accts_raw_df[["DATE", "SC_CODE", "ACCOUNT_CODE"]]

np_cols = np.array(
    [
        "Submitted Date",
        "NP First Name",
        "NP Middle Name",
        "NP Last Name",
        "NP Code",
        "Sponsor Code",
        "Approver Code",
        "TA Promo Date",
        "A Promo Date",
        "AM Promo Date",
        "M Promo Date",
        "SM EXEC Promo Date",
        "AM EXEC Promo Date",
        "DM EXEC Promo Date",
        "D EXEC Promo Date",
    ]
)

# Read NP record file 1
# np_raw_1 = pd.read_excel(r"Provide_NP_Record_2017.xlsx", sheet_name="Marketing_NP")
# np_raw_1_df = pd.DataFrame(np_raw_1, columns=np_cols)

# Read NP record file 2
# np_raw_2 = pd.read_excel(r"Provide_NP_Record_2018.xlsx", sheet_name="Marketing_NP")
# np_raw_2_df = pd.DataFrame(np_raw_2, columns=np_cols)

# Read NP master file
np_raw = pd.read_excel(r"Provide_NP_MasterFile.xlsx", sheet_name="NP")
np_raw_df = pd.DataFrame(np_raw, columns=np_cols)

# Define merged NP dataframe
# np_raw_df = pd.DataFrame(columns=np_cols)
np_raw_df = np_raw_df.astype(
    {
        "Submitted Date": "datetime64",
        "NP First Name": "str",
        "NP Middle Name": "str",
        "NP Last Name": "str",
        "NP Code": "str",
        "Sponsor Code": "str",
        "Approver Code": "str",
        "TA Promo Date": "datetime64",
        "A Promo Date": "datetime64",
        "AM Promo Date": "datetime64",
        "M Promo Date": "datetime64",
        "SM EXEC Promo Date": "datetime64",
        "AM EXEC Promo Date": "datetime64",
        "DM EXEC Promo Date": "datetime64",
        "D EXEC Promo Date": "datetime64",
        "SM EXEC Promo Date": "datetime64",
    },
    errors="ignore",
)

# Merge two NP dataframes together
# np_raw_df = pd.concat([np_raw_1_df, np_raw_2_df])
# np_raw_df = np_raw_df

# Append Provide, Inc. to NP dataframe
np_raw_df = np_raw_df.append(
    pd.Series(
        [
            "2017-01-27",
            "Provide",
            "",
            "Inc.",
            "AA1110",
            "AA1110",
            "AA1110",
            "2017-01-27 00:00:00",
            "2017-01-27 00:00:00",
            "2017-01-27 00:00:00",
            "2017-01-27 00:00:00",
            "2017-01-27 00:00:00",
            "2017-01-27 00:00:00",
            "2017-01-27 00:00:00",
            "2017-01-27 00:00:00",
        ],
        index=np_raw_df.columns,
    ),
    ignore_index=True,
)
np_raw_df.to_excel("temp_np.xlsx")

<IPython.core.display.Javascript object>

In [5]:
np_raw_df["Referrer"] = (
    np_raw_df["NP Last Name"]
    + ", "
    + np_raw_df["NP First Name"]
    + " "
    + np_raw_df["NP Middle Name"].str[:1]
    + "."
)
np_raw_df["TA Promo Date"].replace("NA", np.nan)
np_raw_df["A Promo Date"].replace("NA", np.nan)
np_raw_df["AM Promo Date"].replace("NA", np.nan)
np_raw_df["M Promo Date"].replace("NA", np.nan)
np_raw_df["SM EXEC Promo Date"].replace("NA", np.nan)
np_raw_df["AM EXEC Promo Date"].replace("NA", np.nan)
np_raw_df["DM EXEC Promo Date"].replace("NA", np.nan)
np_raw_df["D EXEC Promo Date"].replace("NA", np.nan)
np_raw_df["NP First Name"].fillna("", inplace=True)
np_raw_df["NP Middle Name"].fillna("", inplace=True)
np_raw_df["NP Last Name"].fillna("", inplace=True)
np_raw_df["NP Code"].fillna("", inplace=True)
np_raw_df["Sponsor Code"].fillna("", inplace=True)
np_raw_df["Approver Code"].fillna("", inplace=True)
np_raw_df.dropna(axis=0, how="all", subset=["Submitted Date"], inplace=True)

str_cols = np.array(
    [
        "NP First Name",
        "NP Middle Name",
        "NP Last Name",
        "NP Code",
        "Sponsor Code",
        "Approver Code",
    ]
)

<IPython.core.display.Javascript object>

In [6]:
for f in range(len(np_raw_df)):
    for g in range(len(str_cols)):
        # print("np_raw_df[str_cols[g]][f]:", f, g, np_raw_df[str_cols[g]][f])
        np_raw_df[str_cols[g]][f].strip()
        np_raw_df[str_cols[g]][f].replace("\n", "")

<IPython.core.display.Javascript object>

In [7]:
cft_df = pd.DataFrame(cft, columns=["CFT Code"])

for g in range(len(cft_df)):
    cft0 = np_raw_df.loc[
        np_raw_df["NP Code"] == cft_df.loc[g, "CFT Code"], "Referrer"
    ].iloc[0]
    cft_df.loc[g, "Referrer"] = cft0

cft_df

,CFT Code,Referrer
0,AA1111,"Valdez, Cindy B."
1,AA1112,"Martinez, Maria Rachel B."
2,AA1117,"Martinez, John Carlos A."
3,AA1120,"Olano, Carmina Angelica V."
4,AA1122,"Ang, John Timothy O."
5,AA1123,"Carmona, Jollie Ann D."
6,AA1127,"Catolico, Cathrine Anne B."
7,AA1128,"Cuesta, Aileen V."
8,AA1130,"Sarmiento, Reychell A."
9,AA1135,"Alvarez, Shiela Q."


<IPython.core.display.Javascript object>

In [8]:
# np_raw_df = pd.merge(np_raw_df, cft_df, left_on="NP Code", right_on="CFT Code", how="left")

np_raw_df["Submitted Date"] = pd.to_datetime(
    np_raw_df["Submitted Date"], errors="coerce"
)
np_raw_df["TA Promo Date"] = pd.to_datetime(np_raw_df["TA Promo Date"], errors="coerce")
np_raw_df["A Promo Date"] = pd.to_datetime(np_raw_df["A Promo Date"], errors="coerce")
np_raw_df["AM Promo Date"] = pd.to_datetime(np_raw_df["AM Promo Date"], errors="coerce")
np_raw_df["M Promo Date"] = pd.to_datetime(np_raw_df["M Promo Date"], errors="coerce")
np_raw_df["SM EXEC Promo Date"] = pd.to_datetime(
    np_raw_df["SM EXEC Promo Date"], errors="coerce"
)
np_raw_df["AM EXEC Promo Date"] = pd.to_datetime(
    np_raw_df["AM EXEC Promo Date"], errors="coerce"
)
np_raw_df["DM EXEC Promo Date"] = pd.to_datetime(
    np_raw_df["DM EXEC Promo Date"], errors="coerce"
)
np_raw_df["D EXEC Promo Date"] = pd.to_datetime(
    np_raw_df["D EXEC Promo Date"], errors="coerce"
)

np_raw_df["NP First Name"] = np_raw_df["NP First Name"].astype(str)
np_raw_df["NP Middle Name"] = np_raw_df["NP Middle Name"].astype(str)
np_raw_df["NP Last Name"] = np_raw_df["NP Last Name"].astype(str)
np_raw_df["NP Code"] = np_raw_df["NP Code"].astype(str)
np_raw_df["Sponsor Code"] = np_raw_df["Sponsor Code"].astype(str)
np_raw_df["Approver Code"] = np_raw_df["Approver Code"].astype(str)
np_raw_df["Submitted Date"] = np_raw_df["Submitted Date"].astype("M8[D]")
np_raw_df["TA Promo Date"] = np_raw_df["TA Promo Date"].astype("M8[D]")
np_raw_df["A Promo Date"] = np_raw_df["A Promo Date"].astype("M8[D]")
np_raw_df["AM Promo Date"] = np_raw_df["AM Promo Date"].astype("M8[D]")
np_raw_df["M Promo Date"] = np_raw_df["M Promo Date"].astype("M8[D]")
np_raw_df["SM EXEC Promo Date"] = np_raw_df["SM EXEC Promo Date"].astype("M8[D]")
np_raw_df["AM EXEC Promo Date"] = np_raw_df["AM EXEC Promo Date"].astype("M8[D]")
np_raw_df["DM EXEC Promo Date"] = np_raw_df["DM EXEC Promo Date"].astype("M8[D]")
np_raw_df["D EXEC Promo Date"] = np_raw_df["D EXEC Promo Date"].astype("M8[D]")

np_raw_df["Sponsor Code"] = np_raw_df["Sponsor Code"].str.strip()
np_raw_df["Approver Code"] = np_raw_df["Approver Code"].str.strip()
# print(np_raw_df.shape)

np_raw_df.dropna(
    axis=0, subset=["Submitted Date", "NP First Name", "NP Last Name"], inplace=True
)
np_raw_df.duplicated(subset=["NP Code"]).sum()
# print(np_raw_df.shape)

np_raw_df.dropna(axis=0, subset=["NP Code"], inplace=True)
# print(np_raw_df.shape)
accts_raw_df.to_excel(r"accts_raw_df.xlsx")
np_raw_df.to_excel(r"np_raw_df.xlsx")

<IPython.core.display.Javascript object>

In [9]:
# Merge Commission, FAMI Accounts, and NP dataframes
fami1 = pd.merge(
    comm_raw_trunc,
    accts_raw_trunc,
    left_on="CUSTOMER_CODE",
    right_on="ACCOUNT_CODE",
    how="inner",
)
# print(fami1.shape)
fami2 = pd.merge(fami1, np_raw_df, left_on="SC_CODE", right_on="NP Code", how="inner")
# print(fami2.shape)
fami2.to_excel(r"fami2_0.xlsx")

# Instantiate a new field
fami2["PROVIDE_RATE"] = np.nan
fami2["PROVIDE_PAYOUT"] = np.nan

# Convert sales load rate to Provide referral rate
for h in range(len(fami2)):
    if calculate_referral:
        if fami2.loc[h, "DATE"] < date_cft:
            fami2.loc[fami2["SALES_LOAD_RATE"] == 2, "PROVIDE_RATE"] = 0.01
            fami2.loc[fami2["SALES_LOAD_RATE"] == 1.5, "PROVIDE_RATE"] = 0.007
            fami2.loc[fami2["SALES_LOAD_RATE"] == 1, "PROVIDE_RATE"] = 0.005
            fami2.loc[fami2["SALES_LOAD_RATE"] == 0.5, "PROVIDE_RATE"] = 0.002
            fami2.loc[fami2["SALES_LOAD_RATE"] == 0, "PROVIDE_RATE"] = 0
            fami2.loc[h, "PROVIDE_PAYOUT"] = (
                fami2.loc[h, "AMOUNTINVESTED"] * fami2.loc[h, "PROVIDE_RATE"] * 0.9
            )
        else:
            fami2.loc[h, "PROVIDE_RATE"] = fami2.loc[h, "SALES_LOAD_RATE"] * 0.9 / 100
            fami2.loc[h, "PROVIDE_PAYOUT"] = (
                fami2.loc[h, "AMOUNTINVESTED"] * fami2.loc[h, "PROVIDE_RATE"]
            )
    else:
        fami2.loc[h, "PROVIDE_RATE"] = 0.006 / 12 * (1 - fami2.loc[h, "WHT_PCT"])
        fami2.loc[h, "PROVIDE_PAYOUT"] = (
            fami2.loc[h, "TRAIL_AMOUNT"] * fami2.loc[h, "PROVIDE_RATE"]
        )

fami2.to_excel(r"fami2.xlsx")

<IPython.core.display.Javascript object>

In [10]:
fami2_ref = fami2[["CUSTOMER_CODE", "SC_CODE"]]
fami2_ref = fami2_ref.groupby(["CUSTOMER_CODE"])["SC_CODE"].nunique()
fami_m = pd.merge(
    fami2, fami2_ref, left_on="CUSTOMER_CODE", right_on="CUSTOMER_CODE", how="inner"
)
fami_m["POS"] = np.nan
fami_m["POSNUM"] = np.nan
fami_m["REF_RT"] = np.nan
fami_m["App No."] = np.nan
fami_m["CFT"] = 0
cft_list = cft_df["CFT Code"].to_list()

for j in range(len(fami_m)):
    fami_m.loc[j, "App No."] = j
    # Check if account has more than one referrer
    num_ref = fami_m.loc[j, "SC_CODE_y"]
    curr_acct = fami_m.loc[j, "CUSTOMER_CODE"] + "_" + fami_m.loc[j, "SECURITY_CODE"]

    # Get referrer's position and corresponding referral rate as of business date
    for i in range(len(position) - 1, -1, -1):
        if fami_m.loc[j, position_promo[i]] is pd.NaT:
            pass
        else:
            if fami_m.loc[j, position_promo[i]] > fami_m.loc[j, "DATE"]:
                pass
            else:
                fami_m.loc[j, "POS"] = position[i]
                fami_m.loc[j, "POSNUM"] = referral_rate["IndexNum"][position][i]
                if fami_m.loc[j, "DATE"] < date_new_rates:
                    fami_m.loc[j, "REF_RT"] = referral_rate["1900-01-01"][position][i]
                else:
                    fami_m.loc[j, "REF_RT"] = referral_rate["2018-07-01"][position][i]
                break

    if fami_m.loc[j, "DATE"] >= date_cft:
        acct = fami_m.loc[j, "CUSTOMER_CODE"] + "_" + fami_m.loc[j, "SECURITY_CODE"]
        if num_ref == 1:
            if fami_m.loc[j, "SC_CODE_x"] in cft_list:
                fami_m.loc[j, "CFT"] = 1
                # Referrer as CFT gets 100% of 70%
                fami_m.loc[j, "REF_RT"] = 0.7
            else:
                fami_m.loc[j, "CFT"] = 0
                # Referrer as non-CFT gets 20% of 70%
                # Remaining 80% of 70% to be allocated to nearest CFT in hierarchy
                fami_m.loc[j, "REF_RT"] = 0.2 * 0.7
        else:
            if curr_acct != acct:
                fami_m.loc[j, "CFT"] = 1
                # Referrer as CFT gets 80% of 70%
                fami_m.loc[j, "REF_RT"] = 0.8 * 0.7
            else:
                # Referrer as non-CFT gets 20% of 70%
                fami_m.loc[j, "REF_RT"] = 0.2 * 0.7
    else:
        if num_ref > 1:
            fami_m.loc[j, "AMOUNTINVESTED"] = fami_m.loc[j, "AMOUNTINVESTED"] / num_ref
            fami_m.loc[j, "TRAIL_AMOUNT"] = fami_m.loc[j, "TRAIL_AMOUNT"] / num_ref
            fami_m.loc[j, "SALES_LOAD_AMOUNT"] = (
                fami_m.loc[j, "SALES_LOAD_AMOUNT"] / num_ref
            )
            fami_m.loc[j, "COMMISSION_LOAD"] = (
                fami_m.loc[j, "COMMISSION_LOAD"] / num_ref
            )
            fami_m.loc[j, "PROVIDE_PAYOUT"] = fami_m.loc[j, "PROVIDE_PAYOUT"] / num_ref
            fami_m.loc[j, "TRAIL_COMM"] = fami_m.loc[j, "TRAIL_COMM"] / num_ref
            fami_m.loc[j, "TOTAL_COMM"] = fami_m.loc[j, "TOTAL_COMM"] / num_ref
            fami_m.loc[j, "TAXABLE_COMM"] = fami_m.loc[j, "TAXABLE_COMM"] / num_ref
            fami_m.loc[j, "WHT"] = fami_m.loc[j, "WHT"] / num_ref
            fami_m.loc[j, "NET_COMM"] = fami_m.loc[j, "NET_COMM"] / num_ref

fami_m.to_excel(r"fami_m.xlsx")

<IPython.core.display.Javascript object>

In [11]:
start = time.process_time()

# Dataframe columns
ref_cols = [
    "DATE",
    "CUSTOMER_CODE",
    "SECURITY_CODE",
    "SC_CODE_x",
    "Referrer",
    "Position",
    "Position Number",
    "Referral Rate",
    "App No.",
    "Sort",
]

# Instantiate temporary dataframes
temp_df = pd.DataFrame(columns=ref_cols)
temp_df2 = pd.DataFrame(columns=ref_cols)
temp_df3 = pd.DataFrame(columns=ref_cols)
temp_df4 = pd.DataFrame(columns=ref_cols)

# Loop through the merged account and referrer dataframe
for j in range(len(fami_m)):
    # Retrieve select columns
    temp_df = temp_df.append(
        {
            "DATE": fami_m.loc[j, "DATE"],
            "CUSTOMER_CODE": fami_m.loc[j, "CUSTOMER_CODE"],
            "SECURITY_CODE": fami_m.loc[j, "SECURITY_CODE"],
            "SC_CODE_x": fami_m.loc[j, "SC_CODE_x"],
            "Referrer": fami_m.loc[j, "Referrer"],
            "Position": fami_m.loc[j, "POS"],
            "Position Number": fami_m.loc[j, "POSNUM"],
            "Referral Rate": fami_m.loc[j, "REF_RT"],
            "App No.": j,
            "Sort": 0,
        },
        ignore_index=True,
    )

    # Assign names to select columns for easier referencing
    bd = temp_df.loc[j, "DATE"]
    cc = temp_df.loc[j, "CUSTOMER_CODE"]
    sc = temp_df.loc[j, "SECURITY_CODE"]
    continue_to_gen = False
    sort = 0
    if temp_df.loc[j, "DATE"] < date_new_rates:
        go_len = 6
        ref_pie = 0.7
        go_pie = 0.3
    elif temp_df.loc[j, "DATE"] >= date_new_rates and temp_df.loc[j, "DATE"] < date_cft:
        go_len = 8
        ref_pie = 0.5
        go_pie = 0.5
    else:
        go_len = 0
        ref_pie = 0
        go_pie = 0

    # Line of sponsorship
    # If referrer is non-manager, calculate referral fees
    if bd < date_cft:
        if temp_df.loc[j, "Position Number"] < 4:
            non_mgr_ref = True
            mgr_hit = False
            prev_pn = temp_df.loc[j, "Position Number"]
            # prev_rr = temp_df.loc[j,'Referral Rate']
            cum_rr = temp_df.loc[j, "Referral Rate"]
            x = 0
            # Get sponsor
            scxa = np_raw_df.loc[
                np_raw_df["NP Code"] == temp_df.loc[j, "SC_CODE_x"], "Sponsor Code"
            ]
            scx1 = scxa.iloc[0]
            while (prev_pn < 4) and (mgr_hit is False):
                # First pass
                if x == 0:
                    scxz = scx1
                # Succeeding passes
                else:
                    scxz0 = np_raw_df.loc[np_raw_df["NP Code"] == scx2, "Sponsor Code"]
                    scxz = scxz0.iloc[0]
                x += 1
                lookup_df2 = np_raw_df.loc[np_raw_df["NP Code"] == scxz, :]
                scxb = np_raw_df.loc[np_raw_df["NP Code"] == scxz, "NP Code"]
                scx2 = scxb.iloc[0]
                nma = np_raw_df.loc[np_raw_df["NP Code"] == scxz, "Referrer"]
                nm1 = nma.iloc[0]
                for i in range(len(position) - 1, -1, -1):
                    if lookup_df2.iloc[0][position_promo[i]] is pd.NaT:
                        pass
                    else:
                        if (
                            lookup_df2.iloc[0][position_promo[i]]
                            > temp_df.loc[j, "DATE"]
                        ):
                            pass
                        else:
                            ps = position[i]
                            pn = referral_rate["IndexNum"][position][i]
                            prev_pn = pn
                            # If sponsor is a manager, exit current loop, proceed to generation override
                            if pn >= 4:
                                mgr_hit = True
                                continue_to_gen = True
                            if temp_df.loc[j, "DATE"] < date_new_rates:
                                rr = referral_rate["1900-01-01"][position][i]
                            else:
                                rr = referral_rate["2018-07-01"][position][i]
                            eff_rr = np.maximum((rr - cum_rr), 0)
                            if eff_rr == 0:
                                break
                            cum_rr = cum_rr + eff_rr
                            sort += 1
                            temp_df2 = temp_df2.append(
                                {
                                    "DATE": bd,
                                    "CUSTOMER_CODE": cc,
                                    "SECURITY_CODE": sc,
                                    "SC_CODE_x": scx2,
                                    "Referrer": nm1,
                                    "Position": ps,
                                    "Position Number": pn,
                                    "Referral Rate": eff_rr,
                                    "App No.": j,
                                    "Sort": sort,
                                },
                                ignore_index=True,
                            )
                            break

        # Generation override
        y = 0
        z = 0
        scxc = np_raw_df.loc[
            np_raw_df["NP Code"] == temp_df.loc[j, "SC_CODE_x"], "Sponsor Code"
        ]
        scx3 = scxc.iloc[0]
        if continue_to_gen is True:
            scxe = np_raw_df.loc[np_raw_df["NP Code"] == scx2, "Sponsor Code"]
            scx5 = scxe.iloc[0]

        l = 0
        cum_rr2 = 0
        while cum_rr2 < go_pie:
            if continue_to_gen is True:
                if z == 0:
                    scxy = scx5
                else:
                    scxy0 = np_raw_df.loc[np_raw_df["NP Code"] == scx4, "Sponsor Code"]
                    scxy = scxy0.iloc[0]
                z += 1
            else:
                if y == 0:
                    scxy = scx3
                else:
                    scxy0 = np_raw_df.loc[np_raw_df["NP Code"] == scx4, "Sponsor Code"]
                    scxy = scxy0.iloc[0]
            y += 1
            lookup_df3 = np_raw_df.loc[np_raw_df["NP Code"] == scxy, :]
            scxd = np_raw_df.loc[np_raw_df["NP Code"] == scxy, "NP Code"]
            scx4 = scxd.iloc[0]
            nmb = np_raw_df.loc[np_raw_df["NP Code"] == scxy, "Referrer"]
            nm2 = nmb.iloc[0]
            for k in range(len(position) - 1, -1, -1):
                if lookup_df3.iloc[0][position_promo][k] is pd.NaT:
                    pass
                else:
                    if scx4 in inactive:
                        rr2 = 0
                        break
                    if lookup_df3.iloc[0][position_promo][k] > temp_df.loc[j, "DATE"]:
                        pass
                    else:
                        ps2 = position[k]
                        pn2 = referral_rate["IndexNum"][position][k]
                        if temp_df.loc[j, "DATE"] < date_new_rates:
                            if pn2 >= 4:
                                rr2 = override_rate["1900-01-01"][l]
                                cum_rr2 = cum_rr2 + rr2
                                l += 1
                            else:
                                rr2 = 0
                                break
                        else:
                            if pn2 >= 4:
                                rr2 = override_rate["2018-07-01"][l]
                                cum_rr2 = cum_rr2 + rr2
                                l += 1
                            else:
                                rr2 = 0
                                break
                    sort += 1
                    temp_df3 = temp_df3.append(
                        {
                            "DATE": bd,
                            "CUSTOMER_CODE": cc,
                            "SECURITY_CODE": sc,
                            "SC_CODE_x": scx4,
                            "Referrer": nm2,
                            "Position": ps2,
                            "Position Number": pn2,
                            "Referral Rate": rr2,
                            "App No.": j,
                            "Sort": sort,
                        },
                        ignore_index=True,
                    )
                    break
    else:
        # Generate CFT, SO, company voucher lines
        scx7 = ""
        nm4 = ""
        ps4 = ""
        ps4 = ""
        ref4 = 0
        w = 0
        # Use net sales load rate instead of Provide referral rate
        # fami_m.loc[j, "PROVIDE_RATE"] = fami_m.loc[j, "SALES_LOAD_RATE"] * 0.9 / 100
        # print("scx7, nm4, ps4, ref4 before while:", scx7, nm4, ps4, ref4)
        if fami_m.loc[j, "CFT"] == 0:
            # Look up in the hierarchy to find Certified Trainer
            # formerly scx6
            scx7 = np_raw_df.loc[
                np_raw_df["NP Code"] == temp_df.loc[j, "SC_CODE_x"], "Sponsor Code"
            ].iloc[0]
            # scx6 = scxf.iloc[0]
            while scx7 not in cft_list:
                temp_runtime = time.process_time() - start
                # print("SC not in CFT?", scx7 not in cft_list)
                # if temp_runtime >= 30:
                #    break
                if w == 0:
                    # scxx = scx6 # old code
                    scxx = scx7
                else:
                    # scxx = np_raw_df.loc[np_raw_df["NP Code"] == scx6, "Sponsor Code"].iloc[0] # old code
                    scxx = np_raw_df.loc[
                        np_raw_df["NP Code"] == scx7, "Sponsor Code"
                    ].iloc[0]
                    # scxx = scxx0.iloc[0]
                w += 1
                # if w > 20:
                #    break
                # print("scxx inside while:", scxx)
                lookup_df4 = np_raw_df.loc[np_raw_df["NP Code"] == scxx, :]
                scx7 = np_raw_df.loc[np_raw_df["NP Code"] == scxx, "NP Code"].iloc[0]
                # scx7 = scxg.iloc[0]
                # print("scx7 inside while:", scx7)
                nm4 = np_raw_df.loc[np_raw_df["NP Code"] == scxx, "Referrer"].iloc[0]
                # nm4 = nmd.iloc[0]
                # print("NP Code, Referrer:", scx7, nm4)
                ref4 = 0.2 * 0.7
                # print("j, w, bd, cc, sc, scx7, nm4, ps4, ref4 inside while:", j, w, bd, cc, sc, scx7, nm4, ps4, ref4)
            else:
                # print("SC not in CFT?", scx7 not in cft_list)
                ref4 = 0.8 * 0.7
                lookup_df4 = np_raw_df.loc[
                    np_raw_df["NP Code"] == scx7, :
                ]  # if sponsor is CFT
                nm4 = np_raw_df.loc[np_raw_df["NP Code"] == scx7, "Referrer"].iloc[0]
                for p in range(len(position) - 1, -1, -1):
                    if lookup_df4.iloc[0][position_promo[p]] is pd.NaT:
                        pass
                    else:
                        if (
                            lookup_df4.iloc[0][position_promo[p]]
                            > temp_df.loc[j, "DATE"]
                        ):
                            pass
                        else:
                            ps4 = position[p]
                            # ps4 = ps4 + " (CFT)"
                            pn4 = referral_rate["IndexNum"][position][p]
                            sort += 1
                            # print("scx7, nm4, ps4, ref4 inside else while:",scx7,nm4,ps4,ref4)
                            temp_df4 = temp_df4.append(
                                {
                                    "DATE": bd,
                                    "CUSTOMER_CODE": cc,
                                    "SECURITY_CODE": sc,
                                    "SC_CODE_x": scx7,
                                    "Referrer": nm4,
                                    "Position": ps4,
                                    "Position Number": pn4,
                                    "Referral Rate": ref4,
                                    "App No.": j,
                                    "Sort": sort,
                                },
                                ignore_index=True,
                            )
                            break
        # Generate voucher lines for SO, company, and BIR
        temp_df4 = temp_df4.append(
            {
                "DATE": bd,
                "CUSTOMER_CODE": cc,
                "SECURITY_CODE": sc,
                "SC_CODE_x": company,
                "Referrer": "Provide, Inc.",
                "Position": "Service Provider",
                "Position Number": 8,
                "Referral Rate": 0.15,
                "App No.": j,
                "Sort": sort + 1,
            },
            ignore_index=True,
        )
        temp_df4 = temp_df4.append(
            {
                "DATE": bd,
                "CUSTOMER_CODE": cc,
                "SECURITY_CODE": sc,
                "SC_CODE_x": so,
                "Referrer": "Catolico, Cathrine Anne B.",
                "Position": "Soliciting Officer",
                "Position Number": 4,
                "Referral Rate": 0.05,
                "App No.": j,
                "Sort": sort + 2,
            },
            ignore_index=True,
        )
        temp_df4 = temp_df4.append(
            {
                "DATE": bd,
                "CUSTOMER_CODE": cc,
                "SECURITY_CODE": sc,
                "SC_CODE_x": so,
                "Referrer": "(Placeholder Account (10% for Tax))",
                "Position": "Tax Collection Agency",
                "Position Number": 8,
                "Referral Rate": 0.1,
                "App No.": j,
                "Sort": sort + 3,
            },
            ignore_index=True,
        )

temp_df = temp_df.append(temp_df2)
temp_df = temp_df.append(temp_df3)
temp_df = temp_df.append(temp_df4)
temp_df.sort_values(by=["App No.", "Sort"], inplace=True)
temp_df.rename(columns={"SC_CODE_x": "SC_CODE"}, inplace=True)
temp_df.to_excel(r"temp_df.xlsx")
runtime = time.process_time() - start
print("Run duration:", runtime)

Run duration: 5.84375


<IPython.core.display.Javascript object>

In [12]:
rf_cols = [
    "Business Date",
    "Client",
    "Fund",
    "Account Number",
    "Member",
    "Position",
    "Investment",
    "Product Rate",
    "Referral Rate",
    "Gross Referral Fee",
    "Withholding Tax",
    "Net Referral Fee",
    "App No.",
    "Sort",
]
trail_cols = [
    "Business Date",
    "Client",
    "Fund",
    "Account Number",
    "Member",
    "Position",
    "Trail Amount",
    "Product Rate",
    "Referral Rate",
    "Gross Referral Fee",
    "Withholding Tax",
    "Net Referral Fee",
    "App No.",
    "Sort",
]
# fami_m0 = fami_m.iloc[:, [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 32, 33, 38]]
fami_m0 = fami_m.loc[
    :,
    [
        "CUSTOMER_CODE",
        "SECURITY_CODE",
        "CLIENT_NAME",
        "SALES_LOAD_RATE",
        "AMOUNTINVESTED",
        "TRAIL_AMOUNT",
        "SALES_LOAD_AMOUNT",
        "COMMISSION_LOAD",
        "TRAIL_COMM",
        "TOTAL_COMM",
        "TAXABLE_COMM",
        "WHT",
        "NET_COMM",
        "PROVIDE_RATE",
        "PROVIDE_PAYOUT",
        "App No.",
    ],
]

fami_m0.to_excel(r"fami_m0.xlsx")
ff = pd.merge(
    fami_m0,
    temp_df,
    left_on=["CUSTOMER_CODE", "SECURITY_CODE", "App No."],
    right_on=["CUSTOMER_CODE", "SECURITY_CODE", "App No."],
    how="inner",
)
ff.to_excel(r"ff0.xlsx")
ff["Gross Referral Fee"] = np.nan
ff["Withholding Tax"] = np.nan
ff["Net Referral Fee"] = np.nan
if calculate_referral:
    ff["Gross Referral Fee"] = (
        ff["AMOUNTINVESTED"] * ff["PROVIDE_RATE"] * ff["Referral Rate"]
    )
    ff["Withholding Tax"] = ff["Gross Referral Fee"] * 0
    ff["Net Referral Fee"] = ff["Gross Referral Fee"] - ff["Withholding Tax"]
else:
    # ff["PROVIDE_RATE"] = 0.006 / 12 * 0.9 # 0.6% / 12 months less 10% WHT
    ff["Gross Referral Fee"] = (
        ff["TRAIL_AMOUNT"] * ff["PROVIDE_RATE"] * ff["Referral Rate"]
    )
    ff["Withholding Tax"] = ff["Gross Referral Fee"] * 0
    ff["Net Referral Fee"] = ff["Gross Referral Fee"] - ff["Withholding Tax"]
ff.rename(
    columns={
        "DATE": "Business Date",
        "CLIENT_NAME": "Client",
        "SECURITY_CODE": "Fund",
        "CUSTOMER_CODE": "Account Number",
        "PROVIDE_RATE": "Product Rate",
        "Referrer": "Member",
        "AMOUNTINVESTED": "Investment",
        "TRAIL_AMOUNT": "Trail Amount",
    },
    inplace=True,
)
ff["Member"].replace("Inc., Provide .", "Provide, Inc.", inplace=True)
ff.sort_values(by=["App No.", "Sort"], inplace=True)

<IPython.core.display.Javascript object>

In [13]:
a = ff.loc[
    (ff["SC_CODE"] == "AA1113") & (ff["Business Date"] >= date_co_sponsorship),
    ["CUSTOMER_CODE", "SECURITY_CODE", "App No."],
]
b = a.index.to_list()
c = [c + 1 for c in b]
d = ff.columns.to_list()
ff2 = pd.DataFrame(columns=d)

# Co-Leadership Agreement
for o in c:
    ff.loc[o, "Referral Rate"] = ff.loc[o, "Referral Rate"] * 0.7
    ff.loc[o, "Gross Referral Fee"] = ff.loc[o, "Gross Referral Fee"] * 0.7
    ff.loc[o, "Withholding Tax"] = ff.loc[o, "Withholding Tax"] * 0.7
    ff.loc[o, "Net Referral Fee"] = ff.loc[o, "Net Referral Fee"] * 0.7
    rr = ff.loc[o, "Referral Rate"] / 0.7 * 0.3
    grf = ff.loc[o, "Gross Referral Fee"] / 0.7 * 0.3
    wt = ff.loc[o, "Withholding Tax"] / 0.7 * 0.3
    nrf = ff.loc[o, "Net Referral Fee"] / 0.7 * 0.3
    ff2 = ff2.append(
        {
            "Account Number": ff.loc[o, "Account Number"],
            "Fund": ff.loc[o, "Fund"],
            "Client": ff.loc[o, "Client"],
            "SALES_LOAD_RATE": ff.loc[o, "SALES_LOAD_RATE"],
            "Product Rate": ff.loc[o, "Product Rate"],
            "Investment": ff.loc[o, "Investment"],
            "Trail Amount": ff.loc[o, "Trail Amount"],
            "SALES_LOAD_AMOUNT": ff.loc[o, "SALES_LOAD_AMOUNT"],
            "COMMISSION_LOAD": ff.loc[o, "COMMISSION_LOAD"],
            "PROVIDE_PAYOUT": ff.loc[o, "PROVIDE_PAYOUT"],
            "TRAIL_COMM": ff.loc[o, "TRAIL_COMM"],
            "TOTAL_COMM": ff.loc[o, "TOTAL_COMM"],
            "TAXABLE_COMM": ff.loc[o, "TAXABLE_COMM"],
            "WHT": ff.loc[o, "WHT"],
            "NET_COMM": ff.loc[o, "NET_COMM"],
            "App No.": ff.loc[o, "App No."],
            "Business Date": ff.loc[o, "Business Date"],
            "SC_CODE": "AA1793",
            "Member": "Peralta, Leah D.",
            "Position": "Manager",
            "Position Number": 4,
            "Referral Rate": rr,
            "Sort": ff.loc[o, "Sort"] + 0.5,
            "Gross Referral Fee": grf,
            "Withholding Tax": wt,
            "Net Referral Fee": nrf,
        },
        ignore_index=True,
    )
ff = ff.append(ff2)
ff.sort_values(
    by=["Account Number", "Fund", "Business Date", "App No.", "Sort"], inplace=True
)
ff.to_excel(r"ff.xlsx")
if calculate_referral:
    ff_cw = pd.DataFrame(ff, columns=rf_cols)
else:
    ff_cw = pd.DataFrame(ff, columns=trail_cols)
    # ff_cw.groupby(["Account Number", "Fund", "Client", "Product Rate", "Referral Rate"],as_index=False,).sum()
ff_cw.to_excel(r"ff_cw.xlsx")

ff_cw2 = ff_cw
ff_cw2["Account"] = np.nan
ff_cw2["Account"] = (
    ff_cw2["Account Number"] + " - " + ff_cw2["Fund"] + " - " + ff_cw2["Client"]
)
acct = ff_cw2["Account"]
ff_cw2.drop(labels=["Account"], axis=1, inplace=True)
ff_cw2.insert(4, "Account", acct)
# if not calculate_referral:
#    ff_cw2.groupby(["Account Number", "Fund", "Client", "Product Rate", "Referral Rate"],as_index=False,).sum()
ff_cw2.to_excel(r"ff_cw2.xlsx")

<IPython.core.display.Javascript object>

In [14]:
if calculate_referral:
    ff_pm = pd.DataFrame(ff, columns=rf_cols)
else:
    ff_pm = pd.DataFrame(ff, columns=trail_cols)
ff_pm.sort_values(by=["Member", "Business Date", "App No.", "Sort"], inplace=True)
# if not calculate_referral:
#    ff_pm.groupby(["Account Number", "Fund", "Client", "Product Rate", "Referral Rate"],as_index=False,).sum()
ff_pm.to_excel(r"ff_pm.xlsx")

<IPython.core.display.Javascript object>